In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

In [ ]:
from ccpi.framework import TestData
from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionData, AcquisitionGeometry
from ccpi.astra.operators import AstraProjectorSimple 

import os, sys

# define some utilities
def get_ideal_sino(N, n_angles):
    # initialise loader
    loader = TestData(data_dir=os.path.join(sys.prefix, 'share','ccpi'))
    data = loader.load(TestData.SIMPLE_PHANTOM_2D, size=(N, N))
    # get ImageGeometry
    ig = data.geometry

    # Create Acquisition data
    angles = np.linspace(0, np.pi, n_angles, dtype=np.float32)

    ag = AcquisitionGeometry(geom_type = 'parallel',
                             dimensions = '2D', 
                             angles = angles,
                             num_pix_h = N)

    dev = 'cpu'

    Aop = AstraProjectorSimple(ig, ag, dev)    
    sino = Aop.direct(data)
    return sino.as_array()

def get_real_sino(N, n_angles):
    
    I_0 = 255
    cor_offset = 10
    
    sino = np.roll(get_ideal_sino(N, n_angles), cor_offset, axis = 1)
    
    flat = np.tile(np.round(I_0-I_0*0.1*np.random.rand(1,N)),(n_angles,1))
    dark = np.tile(np.round(I_0*0.05*np.random.rand(1,N)),(n_angles,1))
    
    flat_noisy = TestData.random_noise(image = flat,
                                       mode = 'poisson',
                                       clip = False)
    
    flat_noisy += dark
    
    proj = flat*np.exp(-sino)
    proj_noisy = TestData.random_noise(image = proj,
                                       mode = 'poisson',
                                       clip = False)
    proj_noisy += dark
    
    return proj_noisy, flat_noisy, dark 
    

## Introduction to reconstruction:
### FBP, CGLS

### Learning objectives
In this notebook you will learn how to set-up analytical (FBP) reconstruction and iterative reconstruction. You will learn the `Algorithm` class, how to set a number of iterations and check intermediate reconstruction results.

### CT data preprocessing

As X-ray photons travel from an X-ray source to detector elements they interact with matter along their trajectories. In these interactions, photons are either absorbed or scattered, resulting in the attenuation of the incident X-ray. A quantitative description of the interaction of X-rays with matter is given by the Beer-Lambert law (or Beer’s law).
$$I^{l} = I^0 \mathrm{exp}\left( -\int_{l} f(g) \mathrm{d}l \right)$$
where $f(g)$ is the X-ray linear attenuation coefficient of the object at the position $g$ along a given linear X-ray trajectory $l$ from the source to the detector element. If $l$ is the entire trajectory from the source to the detector element, then $I^0$ corresponds to the X-ray intensity upon emission from the source and $I^{l}$ corresponds to the X-ray intensity upon incidence on the detector element. $I^{l}$ is typically called a transmission measurement, whereas a projection measurement is given by
$$G^{l} = -\log \left( \frac{I^{l}}{I^0} \right) = \int_{l} f(g) \mathrm{d}l$$

Ideally, $I^0$ is a single value, but real detector pixels do respond equally to photon flux. Secondly, pixels might have residual charge (so called dark current). Therefore, to convert $I^{l}$ to $G^{l}$, one needs to perform flat field correction. If $I^F$ is a flat field image (acquired with source on, without an object in the field of view) and $I^d$ is a dark field image (acquired with source off), then flat field correction is given by:
$$\frac{I-I^D}{I^F-I^D}$$
Let us recall the previous notebook: create `AcquisitionGeometry`, allocate `AcquisitionData`, load sinogram and apply flat and dark-field correction using algebraic operations defined for `DataContainers`. Then, we calculate center of rotation using `CenterOfRotationFinder` and crop the sinogram using `Resizer` to compensate for an offset in the center of rotation.

In [ ]:
# imports
from ccpi.framework import AcquisitionGeometry, AcquisitionData
from ccpi.framework import ImageGeometry, ImageData
from ccpi.processors import CenterOfRotationFinder, Resizer
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# acquisition angles
#n_angles = 90
#angles = np.linspace(0, np.pi, n_angles, dtype = np.float32)

# number of pixels in detector row
#N = 256

# pixel size
#pixel_size_h = 2

# create AcquisitionGeometry
#ag = AcquisitionGeometry(geom_type = 'parallel',
#                         dimension = '2D',
#                         angles = angles,
#                         pixel_num_h = N,
#                         pixel_size_h = pixel_size_h)



In [ ]:
#print('Acquisition geometry:\n{}'.format(ag_par))
#print('Dimension labels:\n{}'.format(ag_par.dimension_labels))

In [ ]:
# allocate AcquisitionData
#ad = ag.allocate()

#print('Dimensions and Labels = {}, {}'.format(ad.shape, ad.dimension_labels))

In [ ]:
# pass actual sinogram to AcquisitionData
# sinogram_tmp, flat_tmp, dark_tmp = get_real_sino(N, n_angles)
# ad_par.fill(sinogram_tmp)

# show sinogram 
# plt.imshow(ad_par.as_array())
# plt.show()

In [ ]:
# load flat and dark field images and wrap them as AcquisitionData objects
#ag_f = ag.clone()
#ag_f.angles = np.array([0], dtype = np.float32)
#ag_d = ag_f.clone()
#flat = ag_f.allocate()
#flat.fill(flat_tmp)
#dark = ag_d.allocate()
#dark.fill(dark_tmp)

# and perform flat field correction and take negative logarithm
#sino_full = -log((ad - dark) / (flat - dark))

#plt.imshow(ad_par.as_array())
#plt.title('raw sino')
#plt.show()

#plt.imshow(sino_full.as_array())
#plt.title('falt-field corrected sinogram')
#plt.show()

In [ ]:
# initialise CenterOfRotationFinder
#cor = CenterOfRotationFinder()
#cor.set_input(sino_full)
#center_of_rotation = cor.get_output()
#print(center_of_rotation)

# initialise Resizer
#resizer_crop = Resizer(binning = [1, 1], roi = [-1, (0, center_of_rotation)])
# pass DataContainer
#resizer_crop.input = sino_full
#sino = resizer_crop.process()
# get new ImageGeometry
#ag_sino = sino.geometry

#plt.imshow(sino_full.as_array())
#plt.title('full sinogram')
#plt.show()

#plt.imshow(sino.as_array())
#plt.title('cropped sinogram')
#plt.show()

In [ ]:
# noise-free sino
#ag_ideal = ag.clone()
#sino_ideal = ag_ideal.allocate()
#sino_ideal.fill(get_ideal_sino(N, n_angles))

#plt.imshow(sino_ideal.as_array())
#plt.title('noise-free sinogram')
#plt.show()

Now the data is ready for reconstruction.

### CT reconstruction
Tomographic reconstruction consists of resolving the three-dimensional photon attenuation map of a scanned object from the collection of projection measurement $G^{l}$. There are two major classes of reconstruction algorithms: *analytic* and *iterative*. 

#### Analytic reconstruction
The most common analytic reconstruction algorithm is filtered back-projection (FBP). The FBP algorithm is derived from the Fourier Slice theorem which relates line integral measurements to two dimensional Fourier transform of an object’s slice. Although the Fourier Slice theorem provides straightforward solution for tomographic reconstruction, its practical implementation is challenging due to required interpolation from Polar to Cartesian coordinates in the Fourier space. In FBP-type reconstruction methods, projections are ﬁltered independently and then back-projected onto the plane of the tomographic slice. Filtration is used to compensate for nonuniform sampling of the Fourier space (higher frequencies have higher density of sampling points) by linear (Ramp) weighting of the frequency space.

In [ ]:
# imports
from ccpi.astra.processors import FBP

#### Iterative reconstruction
Iterative methods use an initial estimate of volume voxel values which is then iteratively updated to best reproduce acquired radiographic data. Since iterative methods involve forward- and back-projection steps, assumptions about data acquisition can be incorporated into the reconstruction procedure. However, iterative methods are computationally demanding, you will notice that it takes much longer to get reconstruction results with iterative methods.

$$\underset{x}{\mathrm{argmin}}\begin{Vmatrix}A x - b\end{Vmatrix}^2_2$$
where,
- $A$ is the projection operator
- $b$ is the acquired data
- $x$ is the solution

In [ ]:
#define the operator A
device = "gpu"
operator = AstraProjectorSimple(ig, ag, device)
    
#setup CGLS
x_init = ig.allocate()
cgls = CGLS(x_init=x_init, operator=operator, data=data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 100

### Summary